In [1]:
import os
import re
import cv2
import pandas as pd
import numpy as np
import json
import csv
import re
import warnings

class Database:
    def __init__(self, videos_path = None, motion_path = None, eegs_path = None, naming_conventions = None):
        self.videos_path = videos_path
        if(type(videos_path) == type(None)):
            self.videos_path = "/data/p_01888/Databook_cleaning/Video/"

        self.motion_path = motion_path
        if(type(motion_path) == type(None)):
            self.motion_path = '/data/pt_01888/motionData/'

        self.eegs_path = motion_path
        if(type(motion_path) == type(None)):
            self.eegs_path = '/data/p_01888/Databook_cleaning/EEG/'

        self.json_filename = "database.json"
        self.csv_filename = "database.csv"
        self.naming_conventions = naming_conventions


    def init_via_videos(self):
        """ Search for videos and append them to a dictionary structure.
            Use this structure as basis for the database.
        """
        if(type(self.videos_path)==type(None)):
            print("Set videospath first")
        self.dictionary = Database.compute_dict(self.videos_path, 
                                                regex_file = self.naming_conventions["regex_video_file"],
                                                regex_folder = self.naming_conventions["regex_video_folder"],
                                                subtree_key = "video")

    def save_as_json(self):
        """ Saves the database dictionary as a .json file """

        with open(self.json_filename, 'w') as outfile:
            json.dump(self.dictionary, outfile)

    def load_json(self, filepath = None):
        """ Loads the database from as .json file """
        if(type(filepath) == type(None)):
            filepath = self.json_filename
        with open(filepath, 'r') as file:
            self.dictionary = json.load(file)

    def save_as_csv(self):
        """ Saves as Comma seperated file """
        dataframe = self.get_dataframe()
        dataframe.to_csv(self.csv_filename, sep='\t', encoding='utf-8')

    @staticmethod
    def n_frames(path):
        """ Obtain number of frames from videos using CV2 """
        cap = cv2.VideoCapture(path)
        frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        return frames

    def get_dict(self):
        """ Getter function for the dictionary that represents the database"""
        return self.dictionary

    def get_keys_of_level(self, lvl = 0, dictionary = None):
        """ Returns all keys for a given level in the dictionary"""
        #We always call with subtrees thus we need subfunction rec_keys_of_level
        if(dictionary==None):
            return self.rec_keys_of_level(self.dictionary, lvl)
        else:
            return self.rec_keys_of_level(dictionary, lvl)

    def rec_keys_of_level(self, subtree,lvl):
        """ Recursive helper method of get_keys_of_level(...)"""
        #if level is 0 return list of get_keys
        if(lvl <= 0):
            ret_lst = []
            try:
                for key, value in subtree.items():
                    ret_lst.append(key)
            except:
                pass
            return ret_lst
        else:
            ret_lst = []
            try:
                for key, value in subtree.items():
                    if(isinstance(value, dict)):
                        ret_lst.extend(self.rec_keys_of_level(value, lvl-1))
            except:
                pass
            return ret_lst

        #otherwise call recursively and return result

    @staticmethod
    def compute_dict(path, regex_file = ".*", regex_folder = ".*", subtree_key = "datatype_x"):
        """ Computes the dictionary by checking for valid video files in path and subdirectories.
            Creates a nested dictionary for all dyads and strores filepathes within.
            Path specifies a folder that contains subfolders
            Only files are processed that match the syntax exemplified by coSMIC_all_P01.wmv
            The dict may be accessed e.g. by returned_dict[0]["video"][2]["path"]
        """
        for root, dirs, files in os.walk(path):#Go through all dirs and files in current directory
            #foldername = root.split(os.sep)[-1] #isolate last part of path
            directories = {}

            sorted_files = [f for f in files]#Sort files by name before beginning
            sorted_files.sort()

            for file in sorted_files:# Go through files and check if they are a valid video
                if(re.match(regex_file, file)):

                    try:
                        attributes = {}
                        attributes["path"] = root + os.sep +file # os.sep equals / or \ respectively on UNIX and Windows
                        
                        camera = re.findall("[0-9]", file)[-1]#Last integer in filename is assumed to specify camera!!
                        directories[camera] = attributes
                    except:
                        warnings.warn("File " + str(file) +" in directory " + str(root+d) +  " didn't match convention" )
                        pass #Regex didn't match any files

            sorted_dirs = [d for d in dirs]# sort directories before beginning
            sorted_dirs.sort()

            for d in sorted_dirs:#Append an int for each dyad
                try:
                    contents = {}
                    if(re.match(regex_folder, d)):
                        try:
                            pair = int("".join(re.findall("[0-9]", d)))
                        except:
                            pass
                        
                        directories[pair] = {}
                        directories[pair][subtree_key] = Database.compute_dict(root+d, regex_file, regex_folder)#Recursive call
                except:
                    warnings.warn("Folder " + str(d) +" in source directory " + str(root+d) +  " didn't match convention" )
                    pass
            return directories
    
    def integrate_framenumbers(self):
        for trial, v in self.dictionary.items():
            for datatype, v1 in self.dictionary[trial].items():
                if datatype == "video":
                    for n_vid, v2 in self.dictionary[trial]["video"].items():
                        try:
                            filepath = self.dictionary[trial]["video"][n_vid]["path"]
                            self.dictionary[trial]["video"][n_vid]["n_frames"] = self.n_frames(filepath)                            
                        except:
                            raise Exception("Couldn't retrieve framenumber for file ")


    def get_dataframe(self):
        """ Get a pandas dataframe representation of the database.
            Convert self.dictionary to a dataframe
        """
        d = self.dictionary
        pairs = [key for key in d.keys()]
        self.un_id = [] # Will store e.g. [['video', 1, 'path'], ['video', 1, 'n_frames']...]

        for value in d.values():#Toplevel will make the rows
            self.acc_rec(value, [])# Retrieve keys to final values in the nested structure e.g [['video', 1, 'path'],...]

        #Make em unique i.e. avoid having same list twice in outer list
        self.un_id = [list(x) for x in set(tuple(i) for i in self.un_id)]
        self.un_id.sort()


        headers = ['.'.join([str(c) for c in x]) for x in self.un_id]#Get a string representation of each sublist

        ndarray = np.ndarray((max(pairs),len(self.un_id)), dtype=object)#Rows x columns

        for y in range(ndarray.shape[0]):
            for x in range(ndarray.shape[1]):
                val = None# FOR DEEPER NESTINGS ADJUST HERE:
                try: # select line by y i.e. first level entry in dict. Within subtree: Get
                    attr = self.un_id[x]

                    if(len(attr)==5):#e.g. ['eeg', 'metainfo', 'type', 48]
                        val = d[y][attr[0]][attr[1]][attr[2]][attr[3]][attr[4]]
                    if(len(attr)==4):#e.g. ['eeg', 'metainfo', 'type', 48]
                        val = d[y][attr[0]][attr[1]][attr[2]][attr[3]]
                    if(len(attr)==3):#e.g.[motion,1,path]
                        val = d[y][attr[0]][attr[1]][attr[2]]
                    if(len(attr)==2):#e.g. ['eeg', 'path']
                        val = d[y][attr[0]][attr[1]]
                    if(len(attr)==1):
                        val = d[y][attr[0]]

                except:# Sometimes values are not present because d[y] is none i.e. pair data is missing
                    ndarray[y][x] = None
                ndarray[y][x] = val

        dataframe = pd.DataFrame(ndarray, columns=headers)

        return dataframe


    def acc_rec(self, node, prefix):
        """
            Appends lists of keys e.g. ['video', 1, 'path'] or ['eeg', 'metainfo', 'channel', 3]
                    to self.un_id recursively
        """
        for key, value in node.items():
            if(isinstance(value, dict)):
                new =[]
                new.extend(prefix)
                new.append(key)
                self.acc_rec(value, new)
            else:
                new = []
                new.extend(prefix)
                new.append(key)
                self.un_id.append(new)

    def integrate_motion(self):
        """
            Check .mot if file exists for given video and naming conventions.
            Add filepath of .mot data.
        """
        try:
            prefix = self.naming_conventions["motion"][0]
            infix = self.naming_conventions["motion"][1]
            postfix = self.naming_conventions["motion"][2]
        except:
            raise ValueError("No (valid) naming convention for 'motion' found. Expected a list of strings for prefix, infix and suffix")
            return
        
        d = self.dictionary
        for pair, v in list(d.items()):#Go down in dict tree (use list when modifying during iteration)
            for video, v1 in list(d[pair].items()):
                for n_video, v2 in list(d[pair][video].items()):
                    
                    path = self.motion_path + prefix + str(pair) + infix + str(n_video)+ postfix                    
                    if(os.path.isfile(path)):
                        self.create_keys(self.dictionary,[pair,video,n_video,"motion","path"])
                        self.dictionary[pair][video][n_video]["motion"]["path"] = path
                                

    def integrate_raw_eegs(self):
        """ Check if EEG files (.eeg and .vmrk) exist for given video and naming conventions.
            Load data if found and add filepath of eeg.
        """
        try:
            eeg_prefix = self.naming_conventions["eeg"][0]
            eeg_suffix = self.naming_conventions["eeg"][1]
            vmrk_prefix = self.naming_conventions["eeg_vmrk"][0]
            vmrk_suffix = self.naming_conventions["eeg_vmrk"][1]

            
        except:
            raise ValueError("No (valid) naming convention for 'eeg' found. Expected a list of strings for prefix and suffix")
            return
        
        d = self.dictionary
        for pair, v in list(d.items()):
            for isvideo, v1 in list(d[pair].items()):
                for n_video, v2 in list(d[pair][isvideo].items()):
                    path = self.eegs_path +eeg_prefix+str(pair)+eeg_suffix
                    path1 = self.eegs_path + vmrk_prefix + str(pair) + vmrk_suffix

                    if(os.path.isfile(path) and os.path.isfile(path1)):
                        self.dictionary[pair]["eeg"] = {}
                        self.dictionary[pair]["eeg"]["path"] = path
                        self.dictionary[pair]["eeg"]["metainfo"] = Database.parse_vmrk(path1)


    search_results = []
    @staticmethod
    def search_key(dictionary, search_key):
        Database.search_results = []
        Database.search_key_rec(dictionary, search_key, path = [])
        return Database.search_results

    @staticmethod
    def search_key_rec(root, search_key, path = []):
        if  isinstance(root, dict):
            for key, value in root.items():
                    current_path = path.copy()
                    current_path.append(key)

                    if key == search_key:
                        Database.search_results.append(current_path)
                    Database.search_key_rec(value, search_key, current_path)


    @staticmethod
    def parse_vmrk(path):
        """ Parses vmrk file and returns a dictionary containing the information.
            The keys denote the kind of data whereas the values are a dictionary
        """

        with open(path) as f:
            content = f.readlines()

        data = {'marker number':[], 'type':[], 'description':[], 'position':[], 'size':[], 'channel':[]}

        entry = 0
        for line in content:
            match = re.match("Mk", line)
            if(match != None):
                markers = re.search("[0-9][0-9]?", line)
                data["marker number"].append(int(markers.group(0)))
                line = line[markers.end():]#use rest of line only next

                markers = re.match("(.*?),",line)
                data["type"].append(markers.group(1)[1:])#Group 1 is exclusive , while group 0 is inclusive ,
                line = line[markers.end():]

                markers = re.search("(.*?),",line)
                data["description"].append(markers.group(1))
                line = line[markers.end():]

                markers = re.search("(.*?),",line)
                data["position"].append('0' + markers.group(1))# '0' + is necessary as some fields are empty
                line = line[markers.end():]

                markers = re.search("(.*?),",line)
                data["size"].append(int('0' + markers.group(1)))
                line = line[markers.end():]

                try:#In the first line there is an additional value we dont want to parse
                    data["channel"].append(int('0' + line))
                except:
                    data["channel"].append(0)
        return data

    def create_keys(self, dictionary, list_of_keys, pos = 0):#e.g. ['1']['motion']['in_roi']['1']
        """
            Creates keys if necessary (path in tree to leave node) such that one may easily add a values (leave)
        """
        if not pos == len(list_of_keys):
            try:#check if exists
                dictionary[list_of_keys[pos]]
            except:
                dictionary[list_of_keys[pos]] = {}

            self.create_keys(dictionary[list_of_keys[pos]], list_of_keys, pos+1)

    @staticmethod
    def max_key(keys):
        """
            Returns the key with the highest value assuming it is an int or string represnentation osf an int
        """
        roi_id = [int(x) for x in keys] #ids will be increasing integers
        roi_id.sort()
        if roi_id == []:#Obtain the highest integer
            current_id = '1'
        else:
            current_id = max(roi_id)
        return current_id

    def add_roi(self, dyad, video_number, start_frame, coordinates, mother = False, child = False, comment = ""):
        """
            Adds the information of a region of interest
        """

        if mother == False and child == False:
            raise ValueError("Either mother or child has to be True")
        if mother == True and child == True:
            raise ValueError("Either mother or child has to be False")

        dyad = str(dyad)
        video_number = str(video_number)

        self.create_keys(self.dictionary,[dyad,'video',video_number,'motion','in_roi'])
        c_id = self.max_key(self.get_dict()[dyad]['video'][video_number]['motion']['in_roi'].keys())

        self.dictionary[dyad]['video'][video_number]['motion']['in_roi'][c_id] = {"start_frame": start_frame,
                                                                        "coordinates" : coordinates,
                                                                        "mother": mother,
                                                                        "child": child,
                                                                        "comment": comment}


In [22]:
Database.compute_dict(path = "/data/p_01888/Databook_cleaning/Video/")
Database.compute_dict(path = "/data/p_01888/Databook_cleaning/Video/", regex_file = "coSMIC_all_P[0-9][0-9]_C[0-9]\\.wmv", regex_folder = "Pilot_[0-9][0-9]")

{1: {'datatype_x': {'1': {'path': '/data/p_01888/Databook_cleaning/Video/Pilot_01/coSMIC_all_P01_C1.wmv'},
   '2': {'path': '/data/p_01888/Databook_cleaning/Video/Pilot_01/coSMIC_all_P01_C2.wmv'},
   '3': {'path': '/data/p_01888/Databook_cleaning/Video/Pilot_01/coSMIC_all_P01_C3.wmv'}}},
 2: {'datatype_x': {'1': {'path': '/data/p_01888/Databook_cleaning/Video/Pilot_02/coSMIC_all_P02_C1.wmv'},
   '2': {'path': '/data/p_01888/Databook_cleaning/Video/Pilot_02/coSMIC_all_P02_C2.wmv'},
   '3': {'path': '/data/p_01888/Databook_cleaning/Video/Pilot_02/coSMIC_all_P02_C3.wmv'}}},
 3: {'datatype_x': {'1': {'path': '/data/p_01888/Databook_cleaning/Video/Pilot_03/coSMIC_all_P03_C1.wmv'},
   '2': {'path': '/data/p_01888/Databook_cleaning/Video/Pilot_03/coSMIC_all_P03_C2.wmv'},
   '3': {'path': '/data/p_01888/Databook_cleaning/Video/Pilot_03/coSMIC_all_P03_C3.wmv'}}},
 4: {'datatype_x': {'1': {'path': '/data/p_01888/Databook_cleaning/Video/Pilot_04/coSMIC_all_P04_C1.wmv'},
   '2': {'path': '/data/p_

In [23]:
naming_conventions ={
                     "video":["coSMIC_all_P","_C",".mov"], 
                     "eeg":["coSMIC_all_P",".eeg"], 
                     "eeg_vmrk":["coSMIC_all_P",".vmrk"],
                     "motion":["coSMIC_all_P","_C",".mot"],
                     "regex_video_file" :"coSMIC_all_P[0-9][0-9]_C[0-9]\\.wmv",
                     "regex_video_folder":"Pilot_[0-9][0-9]"
                    }

In [24]:
#Database.compute_dict('/data/p_01888/Databook_cleaning/Video/')

In [25]:
base = Database(naming_conventions = naming_conventions)

In [26]:
base.init_via_videos()
base.get_dict()

{1: {'video': {'1': {'path': '/data/p_01888/Databook_cleaning/Video/Pilot_01/coSMIC_all_P01_C1.wmv'},
   '2': {'path': '/data/p_01888/Databook_cleaning/Video/Pilot_01/coSMIC_all_P01_C2.wmv'},
   '3': {'path': '/data/p_01888/Databook_cleaning/Video/Pilot_01/coSMIC_all_P01_C3.wmv'}}},
 2: {'video': {'1': {'path': '/data/p_01888/Databook_cleaning/Video/Pilot_02/coSMIC_all_P02_C1.wmv'},
   '2': {'path': '/data/p_01888/Databook_cleaning/Video/Pilot_02/coSMIC_all_P02_C2.wmv'},
   '3': {'path': '/data/p_01888/Databook_cleaning/Video/Pilot_02/coSMIC_all_P02_C3.wmv'}}},
 3: {'video': {'1': {'path': '/data/p_01888/Databook_cleaning/Video/Pilot_03/coSMIC_all_P03_C1.wmv'},
   '2': {'path': '/data/p_01888/Databook_cleaning/Video/Pilot_03/coSMIC_all_P03_C2.wmv'},
   '3': {'path': '/data/p_01888/Databook_cleaning/Video/Pilot_03/coSMIC_all_P03_C3.wmv'}}},
 4: {'video': {'1': {'path': '/data/p_01888/Databook_cleaning/Video/Pilot_04/coSMIC_all_P04_C1.wmv'},
   '2': {'path': '/data/p_01888/Databook_clean

In [27]:
base.integrate_motion()

In [28]:
base.integrate_framenumbers()

In [29]:
base.get_dict()

{1: {'video': {'1': {'n_frames': 21414,
    'path': '/data/p_01888/Databook_cleaning/Video/Pilot_01/coSMIC_all_P01_C1.wmv'},
   '2': {'n_frames': 21417,
    'path': '/data/p_01888/Databook_cleaning/Video/Pilot_01/coSMIC_all_P01_C2.wmv'},
   '3': {'n_frames': 21416,
    'path': '/data/p_01888/Databook_cleaning/Video/Pilot_01/coSMIC_all_P01_C3.wmv'}}},
 2: {'video': {'1': {'n_frames': 12421,
    'path': '/data/p_01888/Databook_cleaning/Video/Pilot_02/coSMIC_all_P02_C1.wmv'},
   '2': {'n_frames': 12421,
    'path': '/data/p_01888/Databook_cleaning/Video/Pilot_02/coSMIC_all_P02_C2.wmv'},
   '3': {'n_frames': 12421,
    'path': '/data/p_01888/Databook_cleaning/Video/Pilot_02/coSMIC_all_P02_C3.wmv'}}},
 3: {'video': {'1': {'n_frames': 47309,
    'path': '/data/p_01888/Databook_cleaning/Video/Pilot_03/coSMIC_all_P03_C1.wmv'},
   '2': {'n_frames': 47307,
    'path': '/data/p_01888/Databook_cleaning/Video/Pilot_03/coSMIC_all_P03_C2.wmv'},
   '3': {'n_frames': 47307,
    'path': '/data/p_01888/D

In [11]:
vmrk = base.parse_vmrk('/data/p_01888/Databook_cleaning/EEG/coSMIC_all_P1.vmrk')

In [12]:
base.integrate_raw_eegs()

In [13]:
print(base.get_dict()[28]["eeg"]["metainfo"]["description"])
#print(base.get_dict()[28]["eeg"]["metainfo"]["position"])
#print(base.get_dict()[28]["video"][2]["n_frames"])

['', 'ControlBox is not connected via USB', 'R128', 'S 13', 'S 10', 'S 11', 'S 40', 'S 41', 'S 40', 'S 43', 'S 40', 'S 41', 'S 40', 'S 42', 'S 40', 'S 41', 'S 40', 'S 46', 'S 40', 'S 41', 'S 40', 'S 45', 'S 40', 'S 41', 'S 40', 'S 49', 'S 40', 'S 41', 'S 40', 'S 48', 'S 10', 'R128']


In [14]:
base.get_dataframe()

,eeg.metainfo.channel,eeg.metainfo.description,eeg.metainfo.marker number,eeg.metainfo.position,eeg.metainfo.size,eeg.metainfo.type,eeg.path,video.1.motion.path,video.1.n_frames,video.1.path,video.2.motion.path,video.2.n_frames,video.2.path,video.3.motion.path,video.3.n_frames,video.3.path,video.4.motion.path,video.4.n_frames,video.4.path
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[, ControlBox is not connected via USB, S 10, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]","[01, 011, 019487, 029007, 032412, 059016, 0822...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[New Segment, Comment, Stimulus, Stimulus, Sti...",/data/p_01888/Databook_cleaning/EEG/coSMIC_all...,None,21414,/data/p_01888/Databook_cleaning/Video/Pilot_01...,None,21417,/data/p_01888/Databook_cleaning/Video/Pilot_01...,None,21416,/data/p_01888/Databook_cleaning/Video/Pilot_01...,None,None,None
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[, ControlBox is not connected via USB, S128, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[01, 01, 015282, 016436, 023716, 053725, 05883...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[New Segment, Comment, Stimulus, Stimulus, Sti...",/data/p_01888/Databook_cleaning/EEG/coSMIC_all...,None,12421,/data/p_01888/Databook_cleaning/Video/Pilot_02...,None,12421,/data/p_01888/Databook_cleaning/Video/Pilot_02...,None,12421,/data/p_01888/Databook_cleaning/Video/Pilot_02...,None,None,None
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[, ControlBox is not connected via USB, S128, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[01, 011, 019262, 027796, 029037, 059029, 0730...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[New Segment, Comment, Stimulus, Stimulus, Sti...",/data/p_01888/Databook_cleaning/EEG/coSMIC_all...,None,47309,/data/p_01888/Databook_cleaning/Video/Pilot_03...,None,47307,/data/p_01888/Databook_cleaning/Video/Pilot_03...,None,47307,/data/p_01888/Databook_cleaning/Video/Pilot_03...,None,None,None
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[, ControlBox is not connected via USB, S128, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[01, 01, 019961, 053450, 058864, 088873, 09787...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[New Segment, Comment, Stimulus, Stimulus, Sti...",/data/p_01888/Databook_cleaning/EEG/coSMIC_all...,None,34718,/data/p_01888/Databook_cleaning/Video/Pilot_04...,None,34716,/data/p_01888/Databook_cleaning/Video/Pilot_04...,None,34717,/data/p_01888/Databook_cleaning/Video/Pilot_04...,None,None,None
5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[, ControlBox is not connected via USB, S128, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[01, 01, 017490, 024050, 027598, 057608, 06181...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[New Segment, Comment, Stimulus, Stimulus, Sti...",/data/p_01888/Databook_cleaning/EEG/coSMIC_all...,None,26955,/data/p_01888/Databook_cleaning/Video/Pilot_05...,None,26956,/data/p_01888/Databook_cleaning/Video/Pilot_05...,None,26956,/data/p_01888/Databook_cleaning/Video/Pilot_05...,None,None,None
6,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[, ControlBox is not connected via USB, S128, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[01, 01, 018122, 020926, 024266, 054276, 08950...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[New Segment, Comment, Stimulus, Stimulus, Sti...",/data/p_01888/Databook_cleaning/EEG/coSMIC_all...,None,22905,/data/p_01888/Databook_cleaning/Video/Pilot_06...,None,22907,/data/p_01888/Databook_cleaning/Video/Pilot_06...,None,22908,/data/p_01888/Databook_cleaning/Video/Pilot_06...,None,None,None
7,None,None,None,None,None,None,None,None,49968,/data/p_01888/Databook_cleaning/Video/Pilot_07...,None,49965,/data/p_01888/Databook_cleaning/Video/Pilot_07...,None,49965,/data/p_01888/Databook_cleaning/Video/Pilot_07...,None,None,None
8,None,None,None,None,None,None,None,None,19819,/data/p_01888/Databook_clea

In [15]:
base.save_as_json()

In [16]:
base.save_as_csv()

In [17]:
base.load_json()#After loading keys are strings

In [18]:
dyad = 33
pos = 10000000000000000
try:
    for key, value in pair[str(dyad)]['eeg']['metainfo']['description'].items():#Search for R128
        if(value == 'R128'):
            newpos = int(pair[str(dyad)]['eeg']['metainfo']['position'][key])
            if(newpos < pos):#find smallest R128 value
                pos = newpos
except:
    pass

In [19]:
base.create_keys(base.dictionary,['1','motion','2','in_roi'])
base.dictionary['1']['motion']['2']['in_roi']

{}

In [20]:
base.add_roi(29,1,1000,[1,2,3,4], mother = True)

In [21]:
base.dictionary['29']['motion']['1']['in_roi']

KeyError: 'motion'

In [ ]:
base.dictionary

In [ ]:
p = {"farneback":{"select trial" :{"define ROI":{"weighted histograms":{"logscale":{}}},
                                    "define tracking ROI":"weighted histograms"}}}

p = base.dictionary

In [ ]:
class Dotstring:
    initial = 0
    full_list_print = False

    def dotstring(node, formated = False, rootname = "root", direction = "right", shape = "box"):
        Dotstring.initial = 0
        header = "graph {"
        footer = "\n}"
        
        if direction == "right":
            direc = "\nrankdir=LR;\n"
        elif direction == "down":
            direc = "\n"
            
        Dotstring.additional_styling = "shape = " + shape + ", "


        connections = []
        ids = ['\n0['+Dotstring.additional_styling+'label = "' + rootname +'"]']
        Dotstring.assemble_rec(node, connections, ids, rootname,0)            

        body = "".join(connections) + "".join(ids)
        
        if not formated:
            body.replace("\n", "")

        return header + direc + body + footer


    def get_current_id():
        Dotstring.initial +=1
        return Dotstring.initial

    def assemble_rec(node, connections, ids, father_name, father_id):
        for k, v in node.items():
            if isinstance(k, str) or isinstance(k,int) or isinstance(key, list):
                child_name = str(k)
                child_id = Dotstring.get_current_id()
                
                connections.append("\n" + str(father_id) + "--" + str(child_id) + ";")
                ids.append("\n"+ str(child_id)+'['+Dotstring.additional_styling+'label = "' + child_name +'"];')
                
            if isinstance(v, str) or isinstance(v,int) or isinstance(v, list):
                grand_child_name = str(v)
                grand_child_id = Dotstring.get_current_id()
                
                if isinstance(v, list) and len(grand_child_name) >50 and not Dotstring.full_list_print:
                    grand_child_name = str(grand_child_name[0:50]) + " ..."
                    
                
                connections.append("\n" + str(child_id) + "--" + str(grand_child_id) + ";")
                ids.append("\n"+ str(grand_child_id)+'['+Dotstring.additional_styling+'label = "' + grand_child_name +'"];')

            if isinstance(v, dict):
                Dotstring.assemble_rec(v, connections, ids, child_name, child_id)
                
                     

In [ ]:
import pydot

dot_string = Dotstring.dotstring(p, rootname = "raw data", direction = "right", shape = "oval" )


graphs = pydot.graph_from_dot_data( dot_string )
svg_string = graphs[0].create_svg() 

In [ ]:
from IPython.display import SVG
from IPython.display import HTML

#print_direc = {'1': base.get_dict()["1"]}
#string = dotstring(print_direc)
#graphs = pydot.graph_from_dot_data(string)
#svg_string = graphs[0].create_svg() 


In [ ]:

display(SVG(svg_string))


HTML("<style>svg{width:50% !important;height:50% !important;}</style>")